# 🔍 Demo 2: Object Detection on Satellite Images

**Using trained YOLO models to detect ships and vehicles**

## What We're Doing:
- Load our fine-tuned YOLO models
- Detect ships at Port of LA
- Detect cars at Mall of America
- Visualize detection results

---

In [ ]:
# Setup - Works both locally and in SageMaker
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Detect environment
IS_SAGEMAKER = os.path.exists('/home/ec2-user/SageMaker') or os.environ.get('SM_MODEL_DIR') is not None

if IS_SAGEMAKER:
    PROJECT_ROOT = Path('/home/ec2-user/SageMaker/Real-Time-Economic-Forecasting')
    USE_S3 = True
    print('\U0001F329\uFE0F  Running in AWS SageMaker')
else:
    PROJECT_ROOT = Path.cwd().parent.parent  # notebooks/demo/ -> project root
    USE_S3 = False
    print('\U0001F4BB Running locally')

# S3 Configuration
S3_RAW = 'economic-forecast-raw'
S3_MODELS = 'economic-forecast-models'
S3_PROCESSED = 'economic-forecast-processed'

# Path helper
def get_path(path_type):
    '''Get path for data - S3 or local based on environment.'''
    paths_s3 = {
        'satellite': f's3://{S3_RAW}/satellite/google_earth',
        'models': f's3://{S3_MODELS}/yolo',
        'ais': f's3://{S3_PROCESSED}/ais',
        'results': f's3://{S3_PROCESSED}/annotations',
    }
    paths_local = {
        'satellite': PROJECT_ROOT / 'data' / 'raw' / 'satellite' / 'google_earth',
        'models': PROJECT_ROOT / 'data' / 'models' / 'satellite',
        'ais': PROJECT_ROOT / 'data' / 'processed' / 'ais',
        'results': PROJECT_ROOT / 'results' / 'annotations',
    }
    return paths_s3.get(path_type) if USE_S3 else paths_local.get(path_type)

print(f'\u2705 Setup complete | S3: {USE_S3}')


---
## 1️⃣ Load Trained Models

In [ ]:
# Load our trained models
print("📥 Loading trained models...")

# Paths to our trained models
port_model_path = PROJECT_ROOT / 'data' / 'models' / 'satellite' / 'dota_yolo11_best.pt'
retail_model_path = PROJECT_ROOT / 'data' / 'models' / 'retail' / 'best.pt'

# Check if models exist, otherwise use base model for demo
if port_model_path.exists():
    port_model = YOLO(str(port_model_path))
    print(f"   ✅ Port model loaded: {port_model_path.name}")
else:
    port_model = YOLO('yolo11n.pt')
    print(f"   ⚠️ Using base model for port detection")

if retail_model_path.exists():
    retail_model = YOLO(str(retail_model_path))
    print(f"   ✅ Retail model loaded: {retail_model_path.name}")
else:
    retail_model = YOLO('yolo11n.pt')
    print(f"   ⚠️ Using base model for retail detection")

print("\n✅ Models ready!")

---
## 2️⃣ Port of LA: Ship Detection

In [ ]:
# Find a sample Port of LA image
port_images_dir = PROJECT_ROOT / 'data' / 'raw' / 'satellite' / 'google_earth' / 'Port_of_LA' / '2024'
port_images = list(port_images_dir.glob('*.jpg'))

if port_images:
    sample_port_image = port_images[0]
    print(f"📸 Sample image: {sample_port_image.name}")
    
    # Display original image
    img = Image.open(sample_port_image)
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    plt.title(f'🚢 Port of LA - Original Image ({sample_port_image.name})', fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.show()
    
    print(f"\n📐 Image size: {img.size}")
else:
    print("⚠️ No port images found. Using placeholder.")
    sample_port_image = None

In [ ]:
# Run detection on Port of LA image
if sample_port_image:
    print("🔍 Running ship detection...")
    
    # Run inference
    results = port_model(str(sample_port_image), conf=0.25)
    
    # Get detection counts
    detections = results[0].boxes
    class_names = results[0].names
    
    print(f"\n📊 DETECTION RESULTS")
    print("="*50)
    print(f"Total objects detected: {len(detections)}")
    
    # Count by class
    class_counts = {}
    for box in detections:
        cls_id = int(box.cls[0])
        cls_name = class_names[cls_id]
        class_counts[cls_name] = class_counts.get(cls_name, 0) + 1
    
    print("\nBy class:")
    for cls, count in sorted(class_counts.items(), key=lambda x: -x[1]):
        print(f"   • {cls}: {count}")
    
    # Display annotated image
    annotated = results[0].plot()
    annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(14, 10))
    plt.imshow(annotated_rgb)
    plt.title(f'🚢 Port of LA - Detected Objects ({len(detections)} total)', fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.show()

---
## 3️⃣ Mall of America: Vehicle Detection

In [ ]:
# Find a sample Mall of America image
mall_images_dir = PROJECT_ROOT / 'data' / 'raw' / 'satellite' / 'google_earth' / 'Mall_of_america' / '2017'
mall_images = list(mall_images_dir.glob('*.jpg'))

if mall_images:
    sample_mall_image = mall_images[0]
    print(f"📸 Sample image: {sample_mall_image.name}")
    
    # Display original image
    img = Image.open(sample_mall_image)
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    plt.title(f'🛒 Mall of America - Original Image ({sample_mall_image.name})', fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.show()
else:
    print("⚠️ No mall images found.")
    sample_mall_image = None

In [ ]:
# Run detection on Mall of America image
if sample_mall_image:
    print("🔍 Running vehicle detection...")
    
    # Run inference
    results = retail_model(str(sample_mall_image), conf=0.25)
    
    # Get detection counts
    detections = results[0].boxes
    class_names = results[0].names
    
    print(f"\n📊 DETECTION RESULTS")
    print("="*50)
    print(f"Total objects detected: {len(detections)}")
    
    # Count by class
    class_counts = {}
    for box in detections:
        cls_id = int(box.cls[0])
        cls_name = class_names[cls_id]
        class_counts[cls_name] = class_counts.get(cls_name, 0) + 1
    
    print("\nBy class:")
    for cls, count in sorted(class_counts.items(), key=lambda x: -x[1]):
        print(f"   • {cls}: {count}")
    
    # Display annotated image
    annotated = results[0].plot()
    annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(14, 10))
    plt.imshow(annotated_rgb)
    plt.title(f'🛒 Mall of America - Detected Vehicles ({len(detections)} total)', fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.show()

---
## 4️⃣ Historical Detection Results

Let's look at our pre-computed detection results across all years:

In [ ]:
# Load historical detection results
port_summary = pd.read_csv(PROJECT_ROOT / 'results' / 'annotations' / 'google_earth_tiled' / 'Port_of_LA' / 'all_years_summary.csv')
mall_summary = pd.read_csv(PROJECT_ROOT / 'results' / 'annotations' / 'retail_tiled' / 'Mall_of_america' / 'all_years_summary.csv')

print("🚢 PORT OF LA - Detection Summary (2017-2024)")
print("="*60)
display(port_summary[['year', 'total_images', 'total_ship', 'total_storage-tank', 'total_harbor']])

print("\n🛒 MALL OF AMERICA - Detection Summary (2017-2024)")
print("="*60)
display(mall_summary[['year', 'total_images', 'total_car', 'total_detections']])

In [ ]:
# Visualize detection trends
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Port of LA ships
ax1 = axes[0]
colors = ['red' if y == 2020 else 'steelblue' for y in port_summary['year']]
ax1.bar(port_summary['year'], port_summary['total_ship'], color=colors, edgecolor='black')
ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Ships Detected', fontsize=12)
ax1.set_title('🚢 Port of LA: Ships Detected by Year', fontsize=14, fontweight='bold')
ax1.axhline(y=port_summary['total_ship'].mean(), color='gray', linestyle='--', alpha=0.7, label='Average')
ax1.legend()

# Annotate 2020
ax1.annotate('COVID\nSurge!', xy=(2020, 222), xytext=(2020.5, 180), fontsize=10, color='red',
            arrowprops=dict(arrowstyle='->', color='red'))

# Mall of America cars per image
ax2 = axes[1]
mall_summary['cars_per_image'] = mall_summary['total_car'] / mall_summary['total_images']
colors = ['red' if y == 2020 else 'green' for y in mall_summary['year']]
ax2.bar(mall_summary['year'], mall_summary['cars_per_image'], color=colors, edgecolor='black')
ax2.set_xlabel('Year', fontsize=12)
ax2.set_ylabel('Cars per Image', fontsize=12)
ax2.set_title('🛒 Mall of America: Cars per Image by Year', fontsize=14, fontweight='bold')
ax2.axhline(y=mall_summary['cars_per_image'].mean(), color='gray', linestyle='--', alpha=0.7, label='Average')
ax2.legend()

# Annotate 2020
ax2.annotate('COVID\nLockdown!', xy=(2020, 47), xytext=(2020.5, 70), fontsize=10, color='red',
            arrowprops=dict(arrowstyle='->', color='red'))

plt.tight_layout()
plt.show()

---
## 5️⃣ Side-by-Side: Original vs Detected

In [ ]:
# Show annotated images from our results
port_annotated_dir = PROJECT_ROOT / 'results' / 'annotations' / 'google_earth_tiled' / 'Port_of_LA' / '2024' / 'annotated'
mall_annotated_dir = PROJECT_ROOT / 'results' / 'annotations' / 'retail_tiled' / 'Mall_of_america' / '2017' / 'annotated'

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Port of LA
port_orig = list((PROJECT_ROOT / 'data' / 'raw' / 'satellite' / 'google_earth' / 'Port_of_LA' / '2024').glob('*.jpg'))
port_annot = list(port_annotated_dir.glob('*_annotated.jpg')) if port_annotated_dir.exists() else []

if port_orig:
    axes[0, 0].imshow(Image.open(port_orig[0]))
    axes[0, 0].set_title('🚢 Port of LA - Original', fontsize=12, fontweight='bold')
    axes[0, 0].axis('off')

if port_annot:
    axes[0, 1].imshow(Image.open(port_annot[0]))
    axes[0, 1].set_title('🚢 Port of LA - Detected', fontsize=12, fontweight='bold')
    axes[0, 1].axis('off')

# Mall of America
mall_orig = list((PROJECT_ROOT / 'data' / 'raw' / 'satellite' / 'google_earth' / 'Mall_of_america' / '2017').glob('*.jpg'))
mall_annot = list(mall_annotated_dir.glob('*_annotated.jpg')) if mall_annotated_dir.exists() else []

if mall_orig:
    axes[1, 0].imshow(Image.open(mall_orig[0]))
    axes[1, 0].set_title('🛒 Mall of America - Original', fontsize=12, fontweight='bold')
    axes[1, 0].axis('off')

if mall_annot:
    axes[1, 1].imshow(Image.open(mall_annot[0]))
    axes[1, 1].set_title('🛒 Mall of America - Detected', fontsize=12, fontweight='bold')
    axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

---
## 📝 Summary

### What We Did:
1. **Loaded** fine-tuned YOLO models
2. **Detected** ships at Port of LA (trade indicator)
3. **Detected** cars at Mall of America (retail indicator)
4. **Visualized** 8 years of detection trends

### Key Findings:
- **2020 Port**: +88% ships (supply chain backlog)
- **2020 Mall**: -38% cars (COVID lockdown)

### Next Step:
→ **Demo 3**: Process AIS ship tracking data

In [ ]:
print("="*60)
print("✅ Demo 2 Complete: Object Detection")
print("="*60)
print("\n➡️  Next: Demo_3_AIS_Data.ipynb")